<a href="https://colab.research.google.com/github/jwang44/Try-colabing-in-colab/blob/main/ECSE01_2_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import scipy.stats
import statistics

## The model

In [3]:
def sigmoid(a):
  return 1/(1+np.exp(-a))

In [4]:
class Logistic_regression():
  def __init__(self,X_train,y_train,learning_rate,X_test,y_test):
    self.X_train = X_train
    self.y_train = y_train
    self.learning_rate = learning_rate
    self.X_test = X_test
    self.y_test = y_test

  def fit(self):
    n,m = np.shape(self.X_train)
    itrnum = 500
    W = np.ones([m+1,itrnum+1])
    e = 0.01        
    der = 0
    for k in range(0,itrnum):
      for i in range(0,n):
        xi = self.X_train[i].T
        x0 = np.array([1])
        xi = np.concatenate((xi, x0),axis = 0)
        yi = self.y_train[i]
        der = der-xi*(yi-sigmoid(np.matmul(W[:,k].T,xi)))
      W[:,k+1] = W[:,k]-self.learning_rate*der
      if (np.linalg.norm(W[:,k+1]-W[:,k]))**2<e:
        break 
    return W[:,k+1]
  
  def predict(self):
    w = self.fit()
    n,m = np.shape(self.X_test)   
    y_predict = np.zeros([n,1])
    for i in range(0,n):
      xi = self.X_test[i].T
      x0 = np.array([1])
      xi = np.concatenate((xi, x0),axis = 0)
      p1 = sigmoid(np.matmul(w.T,xi))
      #p0=1-sigmoid(np.matmul(w.T,xi))
      if p1>=0.5:
        y_predict[i] = 1
      else:
        y_predict[i] = 0
    return y_predict

  '''def Accu_eval(self):
    y_predict = self.predict()
    n,i = np.shape(y_predict)
    err = np.sum(np.abs(y_predict-self.y_test))
    accuracy = 1-err/n
    return accuracy'''
  
  def Accu_eval(self):
    y_predict = self.predict()
    n,j = np.shape(y_predict)
    TP = 0
    FP = 0
    TN = 0
    FN = 0
    for i in range(n):
      if  self.y_test[i]==1 and y_predict[i]==1:
        TP = TP+1
      elif self.y_test[i]==0 and y_predict[i]==0:
        TN = TN+1
      elif self.y_test[i]==0 and y_predict[i]==1:
        FP = FP+1
      elif self.y_test[i]==1 and y_predict[i]==0:
        FN = FN+1
    accuracy = (TP+TN)/(TP+TN+FP+FN)
    precision = TP/(TP+FP)
    recall = TP/(TP+FN)
    F = 2*precision*recall/(precision+recall)
    specificity = TN/(FP+TN)
    FPR = FP/(FP+TN)
    print("accuracy:",accuracy)
    print("precision:",precision)
    print("recall:",recall)
    print("F:",F)
    print("specificity:",specificity)
    print("False Positive Rate:",FPR)
    print("")
    return accuracy
    

## Cross validation

In [5]:
class Cross_validation():
  def __init__(self, k):
    # k: k-fold
    self.k = k

  def prepare_data(self, data):
    # data: np array converted from csv
    np.random.shuffle(data)
    X = data[:, :-1]  # features
    y = data[:, -1]   # labels

    # split data into k equal segments, assign them to train and test later
    Xs = np.array_split(X, self.k, axis=0)
    ys = np.array_split(y, self.k, axis=0)
    return Xs, ys

  def get_accuracy(self, Xs, ys, lr):
    accuracies = []
    for i in range(self.k):
      X_cv = Xs[:] # X_cross_validation
      y_cv = ys[:] # y_cross_validation

      X_test = X_cv.pop(i)
      y_test = y_cv.pop(i)

      X_train = np.concatenate(X_cv)
      y_train = np.concatenate(y_cv)

      logistic_regression = Logistic_regression(X_train, y_train, lr, X_test, y_test)

      print("----------FOLD ", i+1, "----------")
      accuracy = logistic_regression.Accu_eval()
      accuracies.append(accuracy)
    return np.mean(accuracies)
      # this will print the evaluation results

## Dataset

In [6]:
# generate new feature by multiplication and normalize
def newfeature(x,y):
  z=x*y
  norz=scipy.stats.zscore(z, axis=0, ddof=0, nan_policy='propagate')
  return norz

In [7]:
%cd "/content/drive/My Drive/"

/content/drive/My Drive


### Orthopedic Patients Dataset

In [17]:
# convert csv to dataframe
df = pd.read_csv('orthopedic_patients.csv')
original_data = df.to_numpy()

In [9]:
# normalize feature
NorData = scipy.stats.zscore(df.iloc[:,:-1], axis=0, ddof=0, nan_policy='propagate')
# normalized dataset
NorDataset = np.column_stack((NorData, df['Class']))
# np.savetxt('normalized_orthopedic_patients.csv', NorPatientData, delimiter=',')

New feature

In [10]:
# new feature
NewF = newfeature(df.pelvic_incidence, df.sacral_slope)
# normalized dataset with new feature
NorDatasetNew = np.column_stack((NorData,NewF,df['Class']))

### Credit Card Dataset

In [ ]:
# convert csv to dataframe
df = pd.read_csv('creditcard.csv')
# data = df.to_numpy()

In [ ]:
# normalize feature
NorData = scipy.stats.zscore(df.iloc[:,:-1], axis=0, ddof=0, nan_policy='propagate')
# normalized dataset
NorDataset = np.column_stack((NorData,df.iloc[:,-1]))

New feature

In [ ]:
NewF1 = newfeature(df.V3, df.V7)
NewF2 = newfeature(df.V11, df.V12)
NewF3 = newfeature(df.V12, df.V16)
NewF4 = newfeature(df.V16, df.V17)
NewF5 = newfeature(df.V16, df.V18)
NewF6 = newfeature(df.V17, df.V18)
# new feature
NewF = np.column_stack((NewF1,NewF2,NewF3,NewF4,NewF5,NewF6))

In [ ]:
# normalized dataset with new feature
NorDatasetNew = np.column_stack((NorData,NewF,df.iloc[:,-1]))

## Experiment with different features

In [13]:
lr = 0.001
cv = Cross_validation(5) # 5-fold cross-validation
Xs, ys = cv.prepare_data(NorDataset)
print("----------Using normalized features, without new features----------")
accu_avg = cv.get_accuracy(Xs, ys, lr)
print("----------AVERAGE ACCURACY", accu_avg, "----------")
print("\n---------------------------------------------------------------------")


----------Using normalized features, without new features----------
----------FOLD  1 ----------


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  


accuracy: 0.8548387096774194
precision: 0.75
recall: 0.7894736842105263
F: 0.7692307692307692
specificity: 0.8837209302325582
False Positive Rate: 0.11627906976744186

----------FOLD  2 ----------
accuracy: 0.8548387096774194
precision: 0.7368421052631579
recall: 0.7777777777777778
F: 0.7567567567567567
specificity: 0.8863636363636364
False Positive Rate: 0.11363636363636363

----------FOLD  3 ----------
accuracy: 0.8064516129032258
precision: 0.8
recall: 0.5714285714285714
F: 0.6666666666666666
specificity: 0.926829268292683
False Positive Rate: 0.07317073170731707

----------FOLD  4 ----------
accuracy: 0.8709677419354839
precision: 0.9047619047619048
recall: 0.76
F: 0.8260869565217391
specificity: 0.9459459459459459
False Positive Rate: 0.05405405405405406

----------FOLD  5 ----------
accuracy: 0.9516129032258065
precision: 0.8421052631578947
recall: 1.0
F: 0.9142857142857143
specificity: 0.9347826086956522
False Positive Rate: 0.06521739130434782

----------AVERAGE ACCURACY 0.8677

In [15]:
lr = 0.001
cv = Cross_validation(5) # 5-fold cross-validation
Xs, ys = cv.prepare_data(NorDatasetNew)
print("----------Using normalized features, with new features----------")
accu_avg = cv.get_accuracy(Xs, ys, lr)
print("----------AVERAGE ACCURACY", accu_avg, "----------")
print("\n---------------------------------------------------------------------")


----------Using normalized features, with new features----------
----------FOLD  1 ----------


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  


accuracy: 0.8548387096774194
precision: 0.9285714285714286
recall: 0.6190476190476191
F: 0.742857142857143
specificity: 0.975609756097561
False Positive Rate: 0.024390243902439025

----------FOLD  2 ----------
accuracy: 0.8387096774193549
precision: 0.5555555555555556
recall: 0.8333333333333334
F: 0.6666666666666667
specificity: 0.84
False Positive Rate: 0.16

----------FOLD  3 ----------
accuracy: 0.8387096774193549
precision: 0.7666666666666667
recall: 0.8846153846153846
F: 0.8214285714285715
specificity: 0.8055555555555556
False Positive Rate: 0.19444444444444445

----------FOLD  4 ----------
accuracy: 0.8225806451612904
precision: 0.7
recall: 0.7368421052631579
F: 0.717948717948718
specificity: 0.8604651162790697
False Positive Rate: 0.13953488372093023

----------FOLD  5 ----------
accuracy: 0.7419354838709677
precision: 0.6470588235294118
recall: 0.5238095238095238
F: 0.5789473684210527
specificity: 0.8536585365853658
False Positive Rate: 0.14634146341463414

----------AVERAGE AC

## Experiment with different learning rates

In [18]:
lrs = np.logspace(-6, -1, 6) # different learning rates to try
# or we can also try other hyperparameters here
cv = Cross_validation(5) # 5-fold cross-validation
Xs, ys = cv.prepare_data(original_data)
for lr in lrs:
  print("----------LEARNING RATE: ", lr, "----------")
  accu_avg = cv.get_accuracy(Xs, ys, lr)
  print("----------AVERAGE ACCURACY", accu_avg, "----------")
  print("\n---------------------------------------------------------------------\n")


----------LEARNING RATE:  1e-06 ----------
----------FOLD  1 ----------
accuracy: 0.3064516129032258
precision: 0.3064516129032258
recall: 1.0
F: 0.46913580246913583
specificity: 0.0
False Positive Rate: 1.0

----------FOLD  2 ----------
accuracy: 0.27419354838709675
precision: 0.27419354838709675
recall: 1.0
F: 0.430379746835443
specificity: 0.0
False Positive Rate: 1.0

----------FOLD  3 ----------
accuracy: 0.3387096774193548
precision: 0.3387096774193548
recall: 1.0
F: 0.5060240963855421
specificity: 0.0
False Positive Rate: 1.0

----------FOLD  4 ----------
accuracy: 0.3709677419354839
precision: 0.3709677419354839
recall: 1.0
F: 0.5411764705882353
specificity: 0.0
False Positive Rate: 1.0

----------FOLD  5 ----------
accuracy: 0.3064516129032258
precision: 0.3064516129032258
recall: 1.0
F: 0.46913580246913583
specificity: 0.0
False Positive Rate: 1.0

----------AVERAGE ACCURACY 0.31935483870967735 ----------

---------------------------------------------------------------------


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  


accuracy: 0.8548387096774194
precision: 0.6923076923076923
recall: 0.9473684210526315
F: 0.7999999999999999
specificity: 0.813953488372093
False Positive Rate: 0.18604651162790697

----------FOLD  2 ----------
accuracy: 0.7741935483870968
precision: 0.56
recall: 0.8235294117647058
F: 0.6666666666666666
specificity: 0.7555555555555555
False Positive Rate: 0.24444444444444444

----------FOLD  3 ----------
accuracy: 0.7903225806451613
precision: 0.9
recall: 0.42857142857142855
F: 0.5806451612903225
specificity: 0.975609756097561
False Positive Rate: 0.024390243902439025

----------FOLD  4 ----------
accuracy: 0.7903225806451613
precision: 1.0
recall: 0.43478260869565216
F: 0.6060606060606061
specificity: 1.0
False Positive Rate: 0.0

----------FOLD  5 ----------
accuracy: 0.8870967741935484
precision: 0.8
recall: 0.8421052631578947
F: 0.8205128205128205
specificity: 0.9069767441860465
False Positive Rate: 0.09302325581395349

----------AVERAGE ACCURACY 0.8193548387096772 ----------

-----